In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pymedphys.dcm import coords_and_dose_from_dcm
from pymedphys.gamma import gamma_dcm, gamma_shell

In [ ]:
gamma_options = {
    'dcm_ref_filepath': "../../../../Downloads/RD.Ecl_WOmask.dcm",
    'dcm_eval_filepath': "../../../../Downloads/RD.Pin.dcm",
    'dose_percent_threshold': 3,
    'distance_mm_threshold': 3,
    'interp_fraction': 10,
    'max_gamma': np.inf,
    'skip_once_passed': False,
    'random_subset': 2000,
    'mask_evaluation': True
}

gamma_shell_method = gamma_dcm(**gamma_options)

valid_gamma = gamma_shell_method[np.invert(np.isnan(gamma_shell_method))]

In [ ]:
100*np.sum(valid_gamma <= 1) / len(valid_gamma)

In [ ]:
plt.hist(valid_gamma, 50);

In [ ]:
distance_mm_threshold = gamma_options['distance_mm_threshold']
dose_percent_threshold = gamma_options['dose_percent_threshold']

lower_percent_dose_cutoff = 20

normalisation = None

In [ ]:
dcm_ref_filepath = gamma_options['dcm_ref_filepath']
dcm_eval_filepath = gamma_options['dcm_eval_filepath']

coords_reference, dose_reference = coords_and_dose_from_dcm(
    dcm_ref_filepath)
coords_evaluation, dose_evaluation = coords_and_dose_from_dcm(
    dcm_eval_filepath)

# coords_evaluation = [
#     coord[1:-1]
#     for coord in coords_evaluation
# ]

# dose_evaluation = dose_evaluation[1:-1,1:-1,1:-1]

# coords_reference = list(coords_reference)
# coords_reference[0] = coords_reference[0] + 20



In [ ]:
[len(coord) for coord in coords_evaluation]

In [ ]:
coords_reference

In [ ]:
coords_evaluation

In [ ]:
np.mean(dose_reference)

In [ ]:
if normalisation is None:
    normalisation = np.max(dose_reference)

dose_threshold = dose_percent_threshold / 100 * normalisation
lower_dose_cutoff = lower_percent_dose_cutoff / 100 * normalisation

In [ ]:
for _ in range(10):
    lower_dose_cutoff = lower_percent_dose_cutoff / 100 * normalisation
    
    sorted_ref_dose = np.sort(np.ravel(dose_reference))
    sorted_ref_dose = sorted_ref_dose[sorted_ref_dose > lower_dose_cutoff]
    ref_98 = np.floor(len(sorted_ref_dose) * 0.98).astype(int)
    normalisation = sorted_ref_dose[ref_98]
    

normalisation

In [ ]:
dose_threshold = dose_percent_threshold / 100 * normalisation
lower_dose_cutoff = lower_percent_dose_cutoff / 100 * normalisation

In [ ]:
dose_threshold

In [ ]:
lower_dose_cutoff

In [ ]:
distance_step_size = distance_mm_threshold / 5

aligned_gamma = gamma_shell(coords_reference, dose_reference,
                coords_reference, dose_evaluation,
                distance_mm_threshold, dose_threshold,
                lower_dose_cutoff=lower_dose_cutoff, distance_step_size=distance_step_size)

In [ ]:
len(coords_evaluation[2])

In [ ]:
mesh_index = np.meshgrid(*[
    np.arange(len(coord_eval))
    for coord_eval in coords_evaluation
])

eval_index = np.reshape(np.array(mesh_index), (3,-1))
eval_index

In [ ]:
len(coords_evaluation[0])

In [ ]:
len(coords_evaluation[1])

In [ ]:
len(coords_evaluation[2])

In [ ]:
np.shape(dose_evaluation)

In [ ]:
np.shape(eval_index)

In [ ]:
eval_index[0,:]

In [ ]:
# [
#     [
#         in_vicinity[0][point_index_axis == in_vicinity[1]]
#         for point_index_axis, in_vicinity in zip(point_index, all_in_vicinity)
#     ]
#     for point_index in eval_index
# ]

In [ ]:
# [
#     i for i in eval_index
# ]

In [ ]:
# all_in_vicinity[0]

In [ ]:
# all_in_vicinity[1]

In [ ]:
# all_in_vicinity[2]

In [ ]:
# coord_diffs

In [ ]:
coord_diffs = [
    coord_ref[:,None] - coord_eval[None,:]
    for coord_ref, coord_eval in zip(coords_reference, coords_evaluation)
]

all_in_vicinity = [
    np.where(np.abs(diff) < distance_mm_threshold)
    for diff in coord_diffs
]

coord_comparison_in_vicinity = [
    (coord_ref[in_vicinity[0]], coord_eval[in_vicinity[1]])
    for coord_ref, coord_eval, in_vicinity
    in zip(coords_reference, coords_evaluation, all_in_vicinity)
]

# dose_comparison_in_vicinity = [
#     (dose_ref[in_vicinity[0]], dose_eval[in_vicinity[1]])
#     for dose_ref, dose_eval, in_vicinity
#     in zip(dose_reference, dose_evaluation, all_in_vicinity)
# ]

# coords_reference[0][np.where(in_vicinity[0])[0]]

In [ ]:
def create_point_combination(coords):
    mesh_index = np.meshgrid(*coords)
    point_combination = np.reshape(np.array(mesh_index), (3,-1))
    
    return point_combination

In [ ]:
# mesh_index = np.meshgrid(*[
#     np.arange(len(coord_eval))
#     for coord_eval in coords_evaluation
# ])

# eval_index = np.reshape(np.array(mesh_index), (3,-1))
# eval_index

In [ ]:
ref_coord_points = create_point_combination([
    in_vicinity[0] for in_vicinity in all_in_vicinity
])

eval_coord_points = create_point_combination([
    in_vicinity[1] for in_vicinity in all_in_vicinity
])

distances = np.sqrt(np.sum([
    coord_diff[ref_points, eval_points]**2
    for ref_points, eval_points, coord_diff
    in zip(ref_coord_points, eval_coord_points, coord_diffs)
], axis=0))

within_distance_threshold = distances < distance_mm_threshold

distances = distances[within_distance_threshold]
ref_coord_points = ref_coord_points[:, within_distance_threshold]
eval_coord_points = eval_coord_points[:, within_distance_threshold]

In [ ]:
dose_diff = (
    dose_evaluation[
        eval_coord_points[0,:], eval_coord_points[1,:], eval_coord_points[2,:]
    ] - 
    dose_reference[
        ref_coord_points[0,:], ref_coord_points[1,:], ref_coord_points[2,:]
    ]
)

In [ ]:
# plt.plot(dose_diff)

In [ ]:
gamma = np.sqrt((dose_diff / dose_threshold)**2 + (distances / distance_mm_threshold)**2)

In [ ]:
# plt.plot(gamma)

In [ ]:
# plt.plot(distances)

In [ ]:
gamma_pass = gamma < 1
eval_pass = eval_coord_points[:, gamma_pass]

In [ ]:
np.shape(eval_pass)

In [ ]:
eval_pass

In [ ]:
# gamma_pass_array

In [ ]:
# np.mean(gamma_pass_array)

In [ ]:
np.shape(eval_coord_points)

In [ ]:
mesh_index = np.meshgrid(*[
    np.arange(len(coord_eval))
    for coord_eval in coords_evaluation
])

eval_index = np.reshape(np.array(mesh_index), (3,-1))
np.shape(eval_index)

In [ ]:


# convert_to_ravel_index(eval_index)

# np.mean(np.arange(np.shape(eval_index)[1]) == convert_to_ravel_index(eval_index))

In [ ]:
# ravel_index

In [ ]:
[0,1,2]
[0,2,1]
[1,0,2]
[1,2,0]
[2,0,1]
[2,1,0]

In [ ]:
def convert_to_ravel_index(points):
    ravel_index = (
        points[2,:] + 
        (points[2,-1] + 1) * points[1,:] + 
        (points[2,-1] + 1) * (points[1,-1] + 1) * points[0,:])
    
    return ravel_index


ravel_index = convert_to_ravel_index(eval_pass)
gamma_pass_array = np.zeros_like(dose_evaluation).astype(np.bool)

gamma_pass_array = np.ravel(gamma_pass_array)
dose_above_cut_off = np.ravel(dose_evaluation) > lower_dose_cutoff

gamma_pass_array[ravel_index] = True

print(np.mean(gamma_pass_array[dose_above_cut_off]))

gamma_pass_array = np.reshape(gamma_pass_array, np.shape(dose_evaluation))

index = 60
plt.figure()
plt.pcolormesh(gamma_pass_array[index,:,:])

plt.figure()
plt.pcolormesh(dose_evaluation[index,:,:] > lower_dose_cutoff)

plt.figure()
plt.pcolormesh(dose_evaluation[index,:,:])

plt.figure()
plt.pcolormesh(dose_reference[index,:,:])

In [ ]:
index = 60
plt.figure()
plt.pcolormesh(gamma_pass_array[index,:,:])

plt.figure()
plt.pcolormesh(dose_evaluation[index,:,:])

print(coords_evaluation[0][60])

plt.figure()
plt.pcolormesh(dose_reference[index,:,:])

print(coords_reference[0][60])

In [ ]:
np.mean(gamma_pass_array)

In [ ]:
shell_pass = gamma_shell_method < 1

any_pass = gamma_pass_array | shell_pass

In [ ]:
dose_above_cut_off = dose_evaluation > lower_dose_cutoff

np.mean(any_pass[dose_above_cut_off])